In [9]:
import os
import pandas as pd
import polars as pl
import numpy as np
from sklearn.linear_model import LinearRegression
import kaggle_evaluation.default_inference_server


# ==========================================
# 1. 전역 변수 설정 및 모델 학습 (노트북 실행 시 1회만 실행됨)
# ==========================================

# 실제 데이터 로드 (train.csv 경로 확인)
TRAIN_PATH = '/kaggle/input/hull-tactical-market-prediction/train.csv'
TARGET_NAME = 'market_forward_excess_returns'

# 피처 정의 (실제 존재하는 피처를 사용해야 합니다. 예시로 'M'으로 시작하는 첫 번째 컬럼 사용)
# 만약 특정 피처(MOM1 등)를 안다면 직접 리스트로 지정하세요:
features = []
model = None

print("학습 데이터 로드 및 모델 학습 시작...")

if os.path.exists(TRAIN_PATH):
    train_df = pd.read_csv(TRAIN_PATH)
    
    # 피처 자동 선택 (M으로 시작하는 컬럼 중 하나 선택하거나, 없으면 수치형 컬럼 사용)
    candidate_features = [c for c in train_df.columns if c.startswith('M')]
    if candidate_features:
        features = [candidate_features[0]] # 첫 번째 M 피처 사용
    else:
        # 만약 M 피처가 없으면 ID와 타겟을 제외한 첫 번째 컬럼 사용 (비상용)
        features = [c for c in train_df.columns if c not in ['date_id', TARGET_NAME]][0:1]
    
    print(f"사용할 피처: {features}")
    
    # 데이터 전처리
    X_train = train_df[features].fillna(0)
    y_train = train_df[TARGET_NAME]
    
    # 타겟 NaN 제거
    mask = y_train.notna()
    X_train = X_train[mask]
    y_train = y_train[mask]
    
    # 모델 학습
    model = LinearRegression()
    model.fit(X_train, y_train)
    print("모델 학습 완료!")

else:
    print(f"경고: {TRAIN_PATH} 파일이 없습니다. (커밋 시에는 없을 수도 있음)")
    # 오류 방지를 위해 빈 모델 생성 (실제 제출 시에는 파일이 있어야 함)
    model = LinearRegression()
    features = ['dummy'] # 에러 방지용


# ==========================================
# 2. Predict 함수 정의 (서버가 계속 호출함)
# ==========================================

def predict(test: pl.DataFrame) -> pl.DataFrame:
    """
    test: 서버가 주는 현재 시점의 데이터 (Polars DataFrame)
    return: 예측 결과가 담긴 Polars DataFrame
    """
    # 1. Polars를 Pandas로 변환 (Scikit-learn 사용을 위해)
    test_pd = test.to_pandas()
    
    # 2. 피처 추출 및 전처리 (학습 때와 똑같이!)
    # 만약 테스트 데이터에 해당 피처가 없으면 0으로 채움
    X_test = pd.DataFrame()
    for f in features:
        if f in test_pd.columns:
            X_test[f] = test_pd[f].fillna(0)
        else:
            X_test[f] = 0
            
    # 3. 예측 수행
    if model is not None:
        try:
            predicted_returns = model.predict(X_test)
        except:
            # 만약 피처 개수 등이 안 맞아서 에러나면 0.0 예측
            predicted_returns = np.zeros(len(test_pd))
    else:
        predicted_returns = np.zeros(len(test_pd))
        
    # 4. 전략 적용 (양수면 1.0, 아니면 0.0)
    allocations = np.where(predicted_returns > 0, 1.0, 0.0)
    
    # 5. 결과 반환 (반드시 Polars DataFrame이어야 함)
    # date_id는 원본 test 데이터의 것을 그대로 써야 합니다.
    return pl.DataFrame({
        "date_id": test['date_id'],
        "prediction": allocations
    })


# ==========================================
# 3. 서버 실행 (건드리지 말 것)
# ==========================================
inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    # 로컬 게이트웨이: 커밋할 때나 로컬 테스트용으로 작동
    inference_server.run_local_gateway(
        (
            '/kaggle/input/hull-tactical-market-prediction/',
        )
    )

학습 데이터 로드 및 모델 학습 시작...
사용할 피처: ['M1']
모델 학습 완료!
